In [53]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from collections import Counter
from sklearn.preprocessing import StandardScaler

In [176]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

In [55]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/BTC Models/BTC-Hourly.csv')
data = data.iloc[::-1].reset_index(drop=True)
data.index = data.index + 1

In [56]:
data.head()

,unix,date,symbol,open,high,low,close,Volume BTC,Volume USD
1,1526364000,2018-05-15 06:00:00,BTC/USD,8733.86,8796.68,8707.28,8740.99,4906603.14,559.93
2,1526367600,2018-05-15 07:00:00,BTC/USD,8740.99,8766.00,8721.11,8739.00,2390398.89,273.58
3,1526371200,2018-05-15 08:00:00,BTC/USD,8739.00,8750.27,8660.53,8728.49,7986062.84,917.79
4,1526374800,2018-05-15 09:00:00,BTC/USD,8728.49,8754.40,8701.35,8708.32,1593991.98,182.62
5,1526378400,2018-05-15 10:00:00,BTC/USD,8708.32,8865.00,8695.11,8795.90,11101273.74,1260.69


In [57]:
open_prices = list(data["open"])
close_prices = list(data["close"])

In [186]:
class TradeBot:
  def __init__(self, open_prices, model, window_length, expected_percentage_return, expected_trade_duration, sequence_length):
    split = int(len(open_prices)*0.7)
    self.prices_open_train = open_prices[:split]
    self.prices_open_test = open_prices[split:]
    self.model = model
    self.window_length = window_length
    self.expected_percentage_return = expected_percentage_return
    self.expected_trade_duration = expected_trade_duration
    self.sequence_length = sequence_length

  @staticmethod
  def max_min_index_finder(data, l):
    assert l%2 == 1, "code breaks down if l is even"
    index = [0]
    for i in range(l,len(data)+1):
      curl = data[i-l:i]
      if curl.index(max(curl)) == (l-1)/2:
        index.append((2*i-l+1)//2)
      if curl.index(min(curl)) == (l-1)/2:
        index.append((2*i-l+1)//2)
    return index

  @staticmethod
  def data_checkpoints(prices_open, percent, l, n, ind):
    d = int(n/2)
    checkpoints = {}
    for i in ind[:-4]:
      if prices_open[i+d+l]-prices_open[i+d]>prices_open[i+d]*percent/100:
        checkpoints[i] = 1
      else:
        checkpoints[i] = 0
    return checkpoints

  @staticmethod
  def make_data(w_and_b_l, l, ind, checkpoints):
    X_data = []
    y_data = []
    for i in range(l,len(w_and_b_l)-3):
      X_data.append(w_and_b_l[i-l:i])
      y_data.append(checkpoints[ind[i]])
    return np.array(X_data), np.array(y_data)

  @staticmethod
  def split_fit(data, index):
    y_pred = []
    model = LinearRegression()
    w_and_b_l = []
    for i in range(1,len(index)):
      if index[i-1]!=index[i]:
        X = np.array(range(0,index[i]-index[i-1])).reshape(-1,1)
        y = np.array(data[index[i-1]:index[i]])
        model.fit(X,y)
        y_temp = model.predict(X)
        for j in range(len(y_temp)):
          y_pred.append(y_temp[j])
        w_and_b_l.append([model.coef_[0],index[i]-index[i-1]])
    return np.array(y_pred), w_and_b_l

  def train(self, loss, optimizer, metrics, epochs, save_best_model = False):
    self.min_max_indices = TradeBot.max_min_index_finder(self.prices_open_train, self.window_length)
    self.fit_lines, self.w_and_b_l = TradeBot.split_fit(self.prices_open_train, self.min_max_indices)
    self.scaler = StandardScaler()
    self.w_and_b_l_normalized = self.scaler.fit_transform(self.w_and_b_l)
    self.checkpoints = TradeBot.data_checkpoints(self.prices_open_train, self.expected_percentage_return, self.expected_trade_duration, self.window_length, self.min_max_indices)
    self.X_data, self.y_data = TradeBot.make_data(self.w_and_b_l_normalized, self.sequence_length, self.min_max_indices, self.checkpoints)

    q_80 = int(len(self.X_data) * .8)
    q_90 = int(len(self.y_data) * .9)

    self.X_train, self.y_train = np.array(self.X_data[:q_80]), np.array(self.y_data[:q_80])
    self.X_val, self.y_val = np.array(self.X_data[q_80:q_90]), np.array(self.y_data[q_80:q_90])
    self.X_test, self.y_test = np.array(self.X_data[q_90:]), np.array(self.y_data[q_90:])

    if save_best_model:
      checkpoint_callback = ModelCheckpoint(filepath='best_model.h5',
                                        monitor='val_loss',
                                        save_best_only=True)

      self.model.compile(loss=loss,
                        optimizer=optimizer,
                        metrics=metrics)
      self.model.fit(self.X_train, self.y_train, validation_data=(self.X_val, self.y_val), epochs = epochs, callbacks=[checkpoint_callback])

      self.model = tf.keras.models.load_model('best_model.h5')

    else:
      self.model.compile(loss=loss,
                        optimizer=optimizer,
                        metrics=metrics)
      self.model.fit(self.X_train, self.y_train, validation_data=(self.X_val, self.y_val), epochs = epochs)
      self.model.save('model')

  def test(self, test_probability_threshold):
    self.test_probability_threshold = test_probability_threshold
    self.test_probability_predictions = self.model.predict(self.X_test)
    self.test_predictions = self.test_probability_predictions>self.test_probability_threshold
    self.test_predictions.reshape(self.test_predictions.shape[0],)
    self.zero_misclassifications = 0
    self.one_misclassifications = 0
    self.total_ones_predicted = np.sum(self.test_predictions==1)
    for i in range(self.test_probability_predictions.shape[0]):
      if self.test_predictions[i]==1 and self.y_test[i]==0:
        self.zero_misclassifications+=1
      if self.test_predictions[i]==0 and self.y_test[i]==1:
        self.one_misclassifications+=1
    self.total_misclassifications = self.one_misclassifications + self.zero_misclassifications
    self.accuracy = 1 - self.total_misclassifications/self.y_test.shape[0]
    print(f'test size:{self.y_test.shape[0]}')
    print(f'number of ones predicted:{self.total_ones_predicted}')
    print(f'number of zeros misclassified as ones:{self.zero_misclassifications}')
    print(f'number of ones misclassifies as zeros:{self.one_misclassifications}')
    print(f'total number of misclassifications:{self.total_misclassifications}')
    print(f'accuracy:{self.accuracy}')

  @staticmethod
  def backtest_make_data(w_and_b_l, l, ind):
    X_data = []
    y_data = []
    for i in range(l,len(w_and_b_l)-3):
      X_data.append(w_and_b_l[i-l:i])
    return np.array(X_data)

  def backtest(self, stop_loss_percent, take_profit_percent, backtest_probability_threshold, window_length):
    self.backtest_window_length = window_length
    self.backtest_probability_threshold = backtest_probability_threshold
    self.backtest_indices = TradeBot.max_min_index_finder(self.prices_open_test, self.backtest_window_length)
    self.backtest_fit_lines, self.backtest_w_and_b_l = TradeBot.split_fit(self.prices_open_test, self.backtest_indices)
    self.backtest_scaler = StandardScaler()
    self.backtest_w_and_b_l_normalized = self.backtest_scaler.fit_transform(self.backtest_w_and_b_l)
    self.backtest_X_data = TradeBot.backtest_make_data(self.backtest_w_and_b_l_normalized, self.sequence_length, self.backtest_indices)

    tpf = take_profit_percent/100
    slf = -stop_loss_percent/100

    i = self.sequence_length
    n = int(self.backtest_window_length/2)

    self.total_profit = 0
    self.trade_history = []
    self.backtest_predictions = self.model.predict(self.backtest_X_data)
    while i<len(self.backtest_indices)-200:
      buy_price = None
      sell_price = None
      profit = None
      intoTrade = False
      current_prediction = self.backtest_predictions[i-self.backtest_window_length]
      if current_prediction > self.backtest_probability_threshold:
        intoTrade = True
        j = self.backtest_indices[i]
        buy_price = self.prices_open_test[j+n]*current_prediction
        while buy_price*slf<(self.prices_open_test[j+n]*current_prediction-buy_price)<buy_price*tpf:
          j+=1
        sell_price = self.prices_open_test[j+n]*current_prediction
        profit = sell_price-buy_price
        self.total_profit+=profit
        self.trade_history.append(profit)
        while self.backtest_indices[i]<j:
          i+=1
      print(f"interation:{i-10}, intoTrade:{intoTrade}, prediction:{current_prediction}, buyPrice:{buy_price}, sellPrice:{sell_price}, profit:{profit}")
      i+=1
    return self.total_profit, self.trade_history

In [214]:
model1 = tf.keras.Sequential([
    layers.Input((7, 2)),
    layers.LSTM(64, return_sequences = True),
    layers.LSTM(64),# LSTM layer after the Dense layer
    layers.Dense(32, activation='relu'),
    layers.Dense(32, activation='relu'),
    # Add more LSTM layers if needed
    tf.keras.layers.Dense(1, activation='sigmoid')  # Output layer, adjust activation as needed
])

In [224]:
trade_bot = TradeBot(open_prices, model1, 23, 2.5, 35, 7)

In [225]:
trade_bot.train(tf.keras.losses.BinaryCrossentropy(), Adam(learning_rate=0.01), ['mean_absolute_error'], 40)

Epoch 1/40
36/36 [==============================] - 7s 42ms/step - loss: 0.1671 - mean_absolute_error: 0.0694 - val_loss: 0.7632 - val_mean_absolute_error: 0.2364
Epoch 2/40
36/36 [==============================] - 0s 13ms/step - loss: 0.0919 - mean_absolute_error: 0.0485 - val_loss: 1.1910 - val_mean_absolute_error: 0.2571
Epoch 3/40
36/36 [==============================] - 0s 12ms/step - loss: 0.0651 - mean_absolute_error: 0.0319 - val_loss: 1.8832 - val_mean_absolute_error: 0.2774
Epoch 4/40
36/36 [==============================] - 0s 13ms/step - loss: 0.0340 - mean_absolute_error: 0.0138 - val_loss: 2.4929 - val_mean_absolute_error: 0.2976
Epoch 5/40
36/36 [==============================] - 0s 12ms/step - loss: 0.0995 - mean_absolute_error: 0.0398 - val_loss: 1.0590 - val_mean_absolute_error: 0.2588
Epoch 6/40
36/36 [==============================] - 0s 12ms/step - loss: 0.0683 - mean_absolute_error: 0.0374 - val_loss: 1.3786 - val_mean_absolute_error: 0.2322
Epoch 7/40
36/36 [====

In [226]:
trade_bot.test(0.5)

5/5 [==============================] - 1s 5ms/step
test size:142
number of ones predicted:27
number of zeros misclassified as ones:20
number of ones misclassifies as zeros:50
total number of misclassifications:70
accuracy:0.5070422535211268


In [227]:
total_profit, trade_history = trade_bot.backtest(1, 2, 0.5, 23)

19/19 [==============================] - 0s 5ms/step
interation:-3, intoTrade:False, prediction:[0.00010337], buyPrice:None, sellPrice:None, profit:None
interation:-2, intoTrade:False, prediction:[8.5727916e-05], buyPrice:None, sellPrice:None, profit:None
interation:-1, intoTrade:False, prediction:[0.00041102], buyPrice:None, sellPrice:None, profit:None
interation:0, intoTrade:False, prediction:[0.4092542], buyPrice:None, sellPrice:None, profit:None
interation:1, intoTrade:False, prediction:[1.4898095e-13], buyPrice:None, sellPrice:None, profit:None
interation:2, intoTrade:False, prediction:[1.4493571e-12], buyPrice:None, sellPrice:None, profit:None
interation:3, intoTrade:False, prediction:[4.486501e-09], buyPrice:None, sellPrice:None, profit:None
interation:4, intoTrade:False, prediction:[5.4044704e-06], buyPrice:None, sellPrice:None, profit:None
interation:5, intoTrade:False, prediction:[0.00229224], buyPrice:None, sellPrice:None, profit:None
interation:6, intoTrade:False, predictio

In [228]:
print(total_profit)

[3018.5195]
